- 범주형에 대한 가능한 인코딩 
    - 원핫 인코딩
    - 카운트 인코딩
    - 순서형 인코딩
- 모델은 다음의 모델만 사용
    - LogisticRegression
- 스케일링은 minmax 만 사용
    - MinMaxScaler
- 모든 랜덤 요소에 seed 값은 42
    - 검증셋 사이즈는 0.2

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split # 검증셋 사이즈 0.2
from sklearn.linear_model import LogisticRegression # 모델
from sklearn.preprocessing import MinMaxScaler # 이걸로만 스케일링 할것
from sklearn.preprocessing import OneHotEncoder 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/Data/"
SEED = 42

In [ ]:
train_df = pd.read_csv(f"{DATA_PATH}titanic.csv")
test_df = pd.read_csv(f"{DATA_PATH}test.csv")
combine = [train_df,test_df]
train_df.head()

,passengerid,survived,pclass,name,gender,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#수치형 데이터 분포

In [ ]:
train_df.describe()

,passengerid,survived,pclass,age,sibsp,parch,fare
count,1309.000000,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,0.377387,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.484918,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,0.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200


#범주형 데이터의 분포

In [ ]:
train_df.describe(include=['O']) # include=['O'] : Object형의 Dtype를 읽는다.

,name,gender,ticket,cabin,embarked
count,1309,1309,1309,295,1307
unique,1307,2,929,186,3
top,"Connolly, Miss. Kate",male,CA. 2343,C23 C25 C27,S
freq,2,843,11,6,914


# 데이터 전처리

- 적은 특성 제거

In [ ]:
print("Before", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

train_df = train_df.drop(['ticket'], axis=1)
test_df = test_df.drop(['Ticket'], axis=1)
combine = [train_df, test_df]

"After", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape

Before (1309, 12) (418, 11) (1309, 12) (418, 11)


('After', (1309, 11), (418, 10), (1309, 11), (418, 10))

In [ ]:
title = train_df.name.str.extract(' ([A-Za-z]+)\.', expand=False)

In [ ]:
for dataset in combine:
    dataset['Title'] = title

pd.crosstab(train_df['Title'], train_df['gender'])

gender,female,male
Title,,
Capt,0,1
Col,0,4
Countess,1,0
Don,0,1
Dona,1,0
Dr,1,7
Jonkheer,0,1
Lady,1,0
Major,0,2


In [ ]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train_df[['Title', 'survived']].groupby(['Title'], as_index=False).mean()

,Title,survived
0,Master,0.377049
1,Miss,0.791667
2,Mr,0.107001
3,Mrs,0.868687
4,Rare,0.310345


In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train_df

,passengerid,survived,pclass,name,gender,age,sibsp,parch,fare,cabin,embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C85,C,3
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,NaN,S,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C123,S,3
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,NaN,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,8.0500,NaN,S,1
1305,1306,1,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,108.9000,C105,C,5
1306,1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,7.2500,NaN,S,1
1307,1308,0,3,"Ware, Mr. Frederick",male,NaN,0,0,8.0500,NaN,S,1


In [ ]:
train_df = train_df.drop(['name', 'passengerid'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]
train_df.shape, test_df.shape

((1309, 10), (418, 10))

In [ ]:
train_df['family'] = 1 + train_df['sibsp'] + train_df['parch']
train_df.head()

,survived,pclass,gender,age,sibsp,parch,fare,cabin,embarked,Title,family
0,0,3,male,22.0,1,0,7.2500,NaN,S,1,2
1,1,1,female,38.0,1,0,71.2833,C85,C,3,2
2,1,3,female,26.0,0,0,7.9250,NaN,S,2,1
3,1,1,female,35.0,1,0,53.1000,C123,S,3,2
4,0,3,male,35.0,0,0,8.0500,NaN,S,1,1


In [ ]:
# 머신러닝을 위해 해당 컬럼 숫자로 변경
# Label Encode를 이용해 라벨 변경
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train_df['gender'])
train_df['gender'] = le.transform(train_df['gender'])
train_df.head()

,survived,pclass,gender,age,sibsp,parch,fare,cabin,embarked,Title,family
0,0,3,1,22.0,1,0,7.2500,NaN,S,1,2
1,1,1,0,38.0,1,0,71.2833,C85,C,3,2
2,1,3,0,26.0,0,0,7.9250,NaN,S,2,1
3,1,1,0,35.0,1,0,53.1000,C123,S,3,2
4,0,3,1,35.0,0,0,8.0500,NaN,S,1,1


- null 값 채우기

In [ ]:
train_df['embarked'].fillna('S', inplace = True)

In [ ]:
train_df['fare'].fillna(train_df['fare'].dropna().median(), inplace=True)

In [ ]:
age_table = train_df[['pclass', 'gender' , 'age']].groupby(["pclass", "gender"]).mean()
age_table

age
pclass gender           
1      0       37.037594
       1       41.029272
2      0       27.499223
       1       30.815380
3      0       22.185329
       1       25.962264

In [ ]:
def fill_age(person):
    if np.isnan(person['age']):
        return age_table.loc[person['pclass'],person['gender']][0]
    else:
        return person['age']

fill_age(train_df.iloc[1])

38.0

In [ ]:
np.isnan(train_df.iloc[5]['age'])

True

In [ ]:
train_df['age'] = train_df.apply(fill_age, axis=1)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  1309 non-null   int64  
 1   pclass    1309 non-null   int64  
 2   gender    1309 non-null   int64  
 3   age       1309 non-null   float64
 4   sibsp     1309 non-null   int64  
 5   parch     1309 non-null   int64  
 6   fare      1309 non-null   float64
 7   cabin     295 non-null    object 
 8   embarked  1309 non-null   object 
 9   Title     1309 non-null   int64  
 10  family    1309 non-null   int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 112.6+ KB


- 요금 범주 나누기

In [ ]:
train_df['Fare_cat'] = 0
train_df.loc[train_df.fare <= 7.91, 'Fare_cat'] = 0
train_df.loc[(train_df.fare > 7.91) & (train_df.fare <= 14.454), 'Fare_cat'] = 1
train_df.loc[(train_df.fare > 14.454) & (train_df.fare <= 31.0), 'Fare_cat'] = 2
train_df.loc[(train_df.fare > 31.0) & (train_df.fare < 513), 'Fare_cat'] = 3

나이 범주 나누기

In [ ]:
train_df['Age_band'] = 0
train_df.loc[train_df['age'] <= 16, 'Age_band'] = 0
train_df.loc[(train_df['age'] > 16) & (train_df.age <= 32), 'Age_band'] = 1
train_df.loc[(train_df.age > 32) & (train_df.age <= 48), 'Age_band'] = 2
train_df.loc[(train_df.age > 48) & (train_df.age <= 64), 'Age_band'] = 3
train_df.loc[train_df.age > 64, 'Age_band'] = 4

In [ ]:
train_df

,survived,pclass,gender,age,sibsp,parch,fare,cabin,embarked,Title,family,Fare_cat,Age_band
0,0,3,1,22.000000,1,0,7.2500,NaN,S,1,2,0,1
1,1,1,0,38.000000,1,0,71.2833,C85,C,3,2,3,2
2,1,3,0,26.000000,0,0,7.9250,NaN,S,2,1,1,1
3,1,1,0,35.000000,1,0,53.1000,C123,S,3,2,3,2
4,0,3,1,35.000000,0,0,8.0500,NaN,S,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,0,3,1,25.962264,0,0,8.0500,NaN,S,1,1,1,1
1305,1,1,0,39.000000,0,0,108.9000,C105,C,5,1,3,2
1306,0,3,1,38.500000,0,0,7.2500,NaN,S,1,1,0,2
1307,0,3,1,25.962264,0,0,8.0500,NaN,S,1,1,1,1


- 원핫 인코딩 (embarked)

In [ ]:
train_df.select_dtypes("object").nunique()

cabin       186
embarked      3
dtype: int64

In [ ]:
cols = ["pclass","gender","age","sibsp","parch","fare","Title","family","Fare_cat","Age_band"]
features = train_df[cols]
features.head()

,pclass,gender,age,sibsp,parch,fare,Title,family,Fare_cat,Age_band
0,3,1,22.0,1,0,7.2500,1,2,0,1
1,1,0,38.0,1,0,71.2833,3,2,3,2
2,3,0,26.0,0,0,7.9250,2,1,1,1
3,1,0,35.0,1,0,53.1000,3,2,3,2
4,3,1,35.0,0,0,8.0500,1,1,1,2


In [ ]:
from sklearn.preprocessing import OneHotEncoder
oh_cols = ["cabin","embarked"]
enc = OneHotEncoder()

tmp = pd.DataFrame(
    enc.fit_transform(train_df[oh_cols]).toarray(),
    columns = enc.get_feature_names_out()
)

features = pd.concat([features,tmp],axis=1)
features.info(verbose=True, null_counts=True)

In [ ]:
features

,pclass,gender,age,sibsp,parch,fare,Title,family,Fare_cat,Age_band,...,cabin_F2,cabin_F33,cabin_F38,cabin_F4,cabin_G6,cabin_T,cabin_nan,embarked_C,embarked_Q,embarked_S
0,3,1,22.000000,1,0,7.2500,1,2,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,1,0,38.000000,1,0,71.2833,3,2,3,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,0,26.000000,0,0,7.9250,2,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,1,0,35.000000,1,0,53.1000,3,2,3,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,3,1,35.000000,0,0,8.0500,1,1,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,1,25.962264,0,0,8.0500,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1305,1,0,39.000000,0,0,108.9000,5,1,3,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1306,3,1,38.500000,0,0,7.2500,1,1,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1307,3,1,25.962264,0,0,8.0500,1,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [ ]:
features= features.fillna(-1)
features.isnull().sum().sum()

0

In [ ]:
scaler = MinMaxScaler()
x_data = scaler.fit_transform(features)
y_data = train_df["survived"]
x_data , y_data

(array([[1.        , 1.        , 0.27345609, ..., 0.        , 0.        ,
         1.        ],
        [0.        , 0.        , 0.473882  , ..., 1.        , 0.        ,
         0.        ],
        [1.        , 0.        , 0.32356257, ..., 0.        , 0.        ,
         1.        ],
        ...,
        [1.        , 1.        , 0.48014531, ..., 0.        , 0.        ,
         1.        ],
        [1.        , 1.        , 0.32308986, ..., 0.        , 0.        ,
         1.        ],
        [1.        , 1.        , 0.32308986, ..., 1.        , 0.        ,
         0.        ]]), 0       0
 1       1
 2       1
 3       1
 4       0
        ..
 1304    0
 1305    1
 1306    0
 1307    0
 1308    0
 Name: survived, Length: 1309, dtype: int64)

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.2,random_state=SEED)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((1047, 580), (262, 580), (1047,), (262,))

In [ ]:
model = LogisticRegression(random_state=SEED)

model.fit(x_train,y_train)

LogisticRegression(random_state=42)

In [ ]:
pred = model.predict(x_valid)
accuracy_score(y_valid,pred)

0.8473282442748091

In [ ]:
coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(model.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

,Feature,Correlation
0,pclass,7.515235
7,embarked,0.569683
9,family,0.155226
6,cabin,0.075417
11,Age_band,-0.017206
5,fare,-0.061664
3,sibsp,-0.155007
10,Fare_cat,-0.183301
4,parch,-0.199172
8,Title,-0.214835
